# Open Power Systems Data

Data pre-processing for time series blog post - extract hourly data for Germany and resample to daily frequency.

In [1]:
import pandas as pd

In [2]:
# Download hourly data from OPSD website
url = 'https://data.open-power-system-data.org/time_series/2018-06-30/'
datafile = url + 'time_series_60min_singleindex.csv'
df_all = pd.read_csv(datafile, index_col='utc_timestamp', parse_dates=True, low_memory=False)
df_all.head()

,cet_cest_timestamp,AL_load_entsoe_power_statistics,AT_load_entsoe_power_statistics,AT_load_entsoe_transparency,AT_solar_generation_actual,AT_wind_onshore_generation_actual,BA_load_entsoe_power_statistics,BA_load_entsoe_transparency,BE_load_entsoe_power_statistics,BE_load_entsoe_transparency,...,SI_price_day_ahead,SI_solar_generation_actual,SI_wind_onshore_generation_actual,SK_load_entsoe_power_statistics,SK_load_entsoe_transparency,SK_price_day_ahead,SK_solar_generation_actual,TR_load_entsoe_power_statistics,UA_west_load_entsoe_power_statistics,interpolated_values
utc_timestamp,,,,,,,,,,,,,,,,,,,,,
2005-12-31 23:00:00,2006-01-01T00:00:00+0100,NaN,6297.0,NaN,NaN,NaN,1425.0,NaN,9829.0,NaN,...,NaN,NaN,NaN,3332.0,NaN,NaN,NaN,NaN,719.0,NaN
2006-01-01 00:00:00,2006-01-01T01:00:00+0100,NaN,6005.0,NaN,NaN,NaN,1358.0,NaN,9447.0,NaN,...,NaN,NaN,NaN,3165.0,NaN,NaN,NaN,NaN,695.0,NaN
2006-01-01 01:00:00,2006-01-01T02:00:00+0100,NaN,5743.0,NaN,NaN,NaN,1283.0,NaN,9062.0,NaN,...,NaN,NaN,NaN,3040.0,NaN,NaN,NaN,NaN,654.0,NaN
2006-01-01 02:00:00,2006-01-01T03:00:00+0100,NaN,5397.0,NaN,NaN,NaN,1175.0,NaN,8589.0,NaN,...,NaN,NaN,NaN,3021.0,NaN,NaN,NaN,NaN,644.0,NaN
2006-01-01 03:00:00,2006-01-01T04:00:00+0100,NaN,5213.0,NaN,NaN,NaN,1134.0,NaN,8247.0,NaN,...,NaN,NaN,NaN,2959.0,NaN,NaN,NaN,NaN,638.0,NaN


In [3]:
def extract_country(df_all, country_code, year_min=None, year_max=None):
    """Extract data for a single country"""
    
    # List of columns to extract
    columns = [col for col in df_all.columns if col.startswith(country_code)]
    
    # Extract columns and remove country codes from column labels
    columns_map = {col : col[3:] for col in columns}
    df_out = df_all[columns].rename(columns=columns_map)
    
    # Exclude years outside of specified range, if any
    if year_min is not None:
        df_out = df_out[df_out.index.year >= year_min]
    if year_max is not None:
        df_out = df_out[df_out.index.year <= year_max]
        
    return df_out

In [4]:
# Extract data for Germany, for complete years 2006-2017 
# (exclude incomplete years 2005 and 2018)
df_hrly = extract_country(df_all, country_code='DE', year_min=2006, year_max=2017)

# Rename columns for convenience
cols_map = {'load_entsoe_power_statistics' : 'Consumption',
            'wind_generation_actual' : 'Wind',
            'solar_generation_actual' : 'Solar'}
df_hrly = df_hrly[list(cols_map.keys())].rename(columns=cols_map)

# Compute wind + solar generation
df_hrly['Wind+Solar'] = df_hrly[['Wind', 'Solar']].sum(axis=1, skipna=False)

# Convert from MW to GW
df_hrly = df_hrly / 1000
df_hrly.head()

,Consumption,Wind,Solar,Wind+Solar
utc_timestamp,,,,
2006-01-01 00:00:00,47.823,NaN,NaN,NaN
2006-01-01 01:00:00,43.444,NaN,NaN,NaN
2006-01-01 02:00:00,41.432,NaN,NaN,NaN
2006-01-01 03:00:00,40.508,NaN,NaN,NaN
2006-01-01 04:00:00,38.865,NaN,NaN,NaN


In [5]:
# Compute daily totals in GWh
df_daily = df_hrly.resample('D').sum(min_count=24)
df_daily = df_daily.rename_axis('Date')
df_daily.head()

,Consumption,Wind,Solar,Wind+Solar
Date,,,,
2006-01-01,1069.184,NaN,NaN,NaN
2006-01-02,1380.521,NaN,NaN,NaN
2006-01-03,1442.533,NaN,NaN,NaN
2006-01-04,1457.217,NaN,NaN,NaN
2006-01-05,1477.131,NaN,NaN,NaN


In [6]:
df_daily.to_csv('opsd_germany_daily.csv')